# Steady State 1D Diffusion in a Composite Bar using PINNs

This notebook give you a headstart in solving your own Partial Differential Equations (PDEs) using neural networks. Let's quickly recap the theory of PINNs before proceeding. We will embed the physics of the problem in the the neural networks and in such setting, we can use them to approximate the solution to a given differential equation and boundary condition without any training data from other solvers. More specifically, the neural network will be trained to minimize a loss function which is formed using the differential equation and the boundary conditions. If the network is able to minimize this loss then it will in effect solve the given differential equation. More information about the Physics Informed Neural Networks (PINNs) can be found in the [paper](https://www.sciencedirect.com/science/article/pii/S0021999118307125?casa_token=1CnSbVeDwJ0AAAAA:-8a6ZMjO7RgYjFBAxIoVU2sWAdsqFzLRTNHA1BkRryNPXx5Vjc8hCDCkS99gcZR0B1rpa33a30yG) published by Raissi et al. 

In this notebook we will solve the steady 1 dimensional heat transfer in a composite bar. We will use NVIDIA's Modulus library to create the problem setup. You can refer to the [Modulus User Documentation](https://docs.nvidia.com/deeplearning/modulus/index.html) for more examples on solving different types of PDEs using the Modulus library, Modulus APIs, Release notes, etc. 

### Learning Outcomes
1. How to use Modulus to simulate physics problems using PINNs
    1. How to write your own PDEs and formulate the different losses
    2. How to use the Constructive Solid Geometry (CSG) module
2. How to use Modulus to solve a parameterized PDEs
    

## Problem Description

Our aim is to obtain the temperature distribution inside the bar that is made up of two materials with different thermal conductivity. The geometry and the problem specification of the problem can be seen below

<img src="diffusion_bar_geometry.png" alt="Drawing" style="width: 600px;"/>

The composite bar extends from $x=0$ to $x=2$. The bar has material of conductivity $D_1=10$ from $x=0$ to $x=1$ and $D_2=0.1$ from $x=1$ to $x=2$. Both the ends of the bar, $x=0$ and $x=2$ are maintained at a constant temperatures of $0$ and $100$ respectively. For simplicity of modeling, we will treat the composite bar as two separate bars, bar 1 and bar 2, whose ends are joined together. We will treat the temperatures in the bar 1 as $U_1$ and the temperature in bar 2 as $U_2$. 

The equations and boundary conditions governing the problem can be mathematically expressed as

One dimensional diffusion of temperature in bar 1 and 2:

$$
\begin{align}
\frac{d}{dx}\left( D_1\frac{dU_1}{dx} \right) = 0, && \text{when } 0<x<1 \\
\frac{d}{dx}\left( D_2\frac{dU_2}{dx} \right) = 0, && \text{when } 1<x<2 \\
\end{align}
$$

Flux and temperature continuity at interface $(x=1)$
$$
\begin{align}
D_1\frac{dU_1}{dx} = D_1\frac{dU_1}{dx}, && \text{when } x=1 \\
U_1 = U_2, && \text{when } x=1 \\
\end{align}
$$


## Case Setup

Now that we have our problem defined, let's take a look at the code required to solve it using Modulus. Modulus has a variety of helper functions/APIs that will help us to set up the problem with ease. It has APIs to model geometry in a parameterized fashion using the Constructive Solid Geometry (CSG) module, write-up the required equations in a user-friendly symbolic format and comes with several advanced neural network architectures to choose for more complicated problems. 

Now let's start the problem by importing the required libraries and packages

### Note

In this notebook we will describe the contents of the [`diffusion_bar.py`](../../source_code/diffusion_1d/diffusion_bar.py) script

```python
import torch
import numpy as np
from sympy import Symbol, Eq, Function, Number

import modulus
from modulus.hydra import ModulusConfig, instantiate_arch
from modulus.solver import Solver
from modulus.domain import Domain
from modulus.geometry.primitives_1d import Line1D
from modulus.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
)
from modulus.domain.validator import PointwiseValidator
from modulus.domain.monitor import PointwiseMonitor
from modulus.key import Key
from modulus.node import Node
from modulus.eq.pde import PDE
```

From v22.03 modulus supports APIs for solving data-driven problems where the data is available on grid/mesh as well as the PINN problems where point clouds are used for problem formulation. In this section, since we will be solving the 1d diffusion problem using the PINN approach, we will use the functions and classes from the `modulus.solver` module. The `Solver` class trains and evaluates the Modulus' neural network solver. The `Domain` class is used to add constraints, monitors, validators, etc. to the problem.

The `geometry` module contains predefined geometries respectively that one can use to define the problem. We will describe each of them in detail as we move forward in the code. For more detailed information on all the different modules, we recommended you to refer the [Modulus API Documentation](https://docs.nvidia.com/deeplearning/modulus/api_source/api_index.html).


## Creating the geometry

In this problem, we will create the 1-dimensional geometry using `Line1D` class from the geometry module. The module also contains several 2d and 3d shapes like rectangle, circle, triangle, cuboids, sphere, torus, cones, tetrahedrons, etc. We will define the one dimensional line object using the two end-points. For composite bar, we will create two separate bars as defined in the problem statement

```python
# params for domain
L1 = Line1D(0,1)
L2 = Line1D(1,2)
```

Next we will define the properties for the problem which will later be used while making the equations, boundary conditions, etc. Also, for this problem, we can find the temperature at the interface analytically and we will use that as to form validation data to compare our neural network results 

```python
D1 = 1e1
D2 = 1e-1

Tc = 100
Ta = 0
Tb = (Tc + (D1 / D2) * Ta) / (1 + (D1 / D2))

print(Ta)
print(Tb)
print(Tc)
```

## Defining the differential equations for the problem

The `PDE` class allows us to write the equations symbolically in Sympy. This allows you to quickly write your equations in the most natural way possible. The Sympy equations are converted to PyTorch expressions in the back-end and can also be printed to ensure correct implementation.

Modulus also comes with several common PDEs predefined for the user to choose from. Some of the PDEs that are already available in the PDEs module are: Navier Stokes, Linear Elasticity, Advection Diffusion, Wave Equations, etc.

Let's create the PDE to define the diffusion equation. We will define the equation in its most generic, transient 3-dimensional, form and then have an argument `dim` that can reduce it to lower dimensional forms. 
$$\frac{\partial T}{\partial t}= \nabla\cdot \left( D \nabla T \right) + Q$$

Let's start defining the equation by inhereting from the `PDE` class. We will create the initialization method for this class that defines the equation(s) of interest. We will be defining the diffusion equation using the source(`Q`), diffusivity(`D`), symbol for diffusion(`T`). If `D` or `Q` is given as a string we will convert it to functional form. This will allow us to solve problems with spatially/temporally varying properties. 

```python
class Diffusion(PDE):
    name = "Diffusion"

    def __init__(self, T="T", D="D", Q=0, dim=3, time=True):
        # set params
        self.T = T
        self.dim = dim
        self.time = time

        # coordinates
        x, y, z = Symbol("x"), Symbol("y"), Symbol("z")

        # time
        t = Symbol("t")

        # make input variables
        input_variables = {"x": x, "y": y, "z": z, "t": t}
        if self.dim == 1:
            input_variables.pop("y")
            input_variables.pop("z")
        elif self.dim == 2:
            input_variables.pop("z")
        if not self.time:
            input_variables.pop("t")

        # Temperature
        assert type(T) == str, "T needs to be string"
        T = Function(T)(*input_variables)

        # Diffusivity
        if type(D) is str:
            D = Function(D)(*input_variables)
        elif type(D) in [float, int]:
            D = Number(D)

        # Source
        if type(Q) is str:
            Q = Function(Q)(*input_variables)
        elif type(Q) in [float, int]:
            Q = Number(Q)

        # set equations
        self.equations = {}
        self.equations["diffusion_" + self.T] = (
            T.diff(t)
            - (D * T.diff(x)).diff(x)
            - (D * T.diff(y)).diff(y)
            - (D * T.diff(z)).diff(z)
            - Q
        )
```

First we defined the input variables $x, y, z$ and $t$ with Sympy symbols. Then we defined the functions for $T$, $D$ and $Q$ that are dependent on the input variables $(x, y, z, t)$. Using these we can write out our simple equation $T_t = \nabla \cdot (D \nabla T) + Q$. We store this equation in the class by adding it to the dictionary of `equations`.

Note that we moved all the terms of the PDE either to LHS or RHS. This way, while using the equations in the constraints, we
can assign a custom source function to the `’diffusion_T’` key instead of 0 to add more source terms to our PDE.

Great! We just wrote our own PDE in Modulus! Once you have understood the process to code a simple PDE, you can easily extend the procedure for different PDEs. You can also bundle multiple PDEs together in a same file by adding new keys to the equations dictionary. Below we show the code for the interface boundary condition where we need to maintain the field (dirichlet) and flux (neumann) continuity. *(More examples of coding your own PDE can be found in the [Modulus User Documentation](https://docs.nvidia.com/deeplearning/modulus/text/foundational/1d_wave_equation.html))*. 

**Note :** The field continuity condition is needed because we are solving for two different temperatures in the two bars. 

```python
class DiffusionInterface(PDE):
    name = "DiffusionInterface"

    def __init__(self, T_1, T_2, D_1, D_2, dim=3, time=True):
        # set params
        self.T_1 = T_1
        self.T_2 = T_2
        self.dim = dim
        self.time = time

        # coordinates
        x, y, z = Symbol("x"), Symbol("y"), Symbol("z")
        normal_x, normal_y, normal_z = (
            Symbol("normal_x"),
            Symbol("normal_y"),
            Symbol("normal_z"),
        )

        # time
        t = Symbol("t")

        # make input variables
        input_variables = {"x": x, "y": y, "z": z, "t": t}
        if self.dim == 1:
            input_variables.pop("y")
            input_variables.pop("z")
        elif self.dim == 2:
            input_variables.pop("z")
        if not self.time:
            input_variables.pop("t")

        # Diffusivity
        if type(D_1) is str:
            D_1 = Function(D_1)(*input_variables)
        elif type(D_1) in [float, int]:
            D_1 = Number(D_1)
        if type(D_2) is str:
            D_2 = Function(D_2)(*input_variables)
        elif type(D_2) in [float, int]:
            D_2 = Number(D_2)

        # variables to match the boundary conditions (example Temperature)
        T_1 = Function(T_1)(*input_variables)
        T_2 = Function(T_2)(*input_variables)

        # set equations
        self.equations = {}
        self.equations["diffusion_interface_dirichlet_" + self.T_1 + "_" + self.T_2] = (
            T_1 - T_2
        )
        flux_1 = D_1 * (
            normal_x * T_1.diff(x) + normal_y * T_1.diff(y) + normal_z * T_1.diff(z)
        )
        flux_2 = D_2 * (
            normal_x * T_2.diff(x) + normal_y * T_2.diff(y) + normal_z * T_2.diff(z)
        )
        self.equations["diffusion_interface_neumann_" + self.T_1 + "_" + self.T_2] = (
            flux_1 - flux_2
        )
```

## Creating PDE and Neural Network nodes

The default `FullyConnectedArch` (selected by setting `cfg.arch.fully_connected` in the `cfg` argument of the `instantiate_arch()` function) can be used to create the neural network for the problem. The default `FullyConnectedArch` represents a 6 layer MLP (multi-layer perceptron) architecture with each layer containing 512 nodes and uses swish as the activation function. All these parameters are user configurable, and can be configured using Hydra config files. Once all the PDEs and architectures are defined, we can create a list of nodes to pass to different constraints we want to satisfy for this problem (i.e. equations residuals, boundary conditions, etc.).

We will define all the code for the problem in the `run` function as shown below. 

```python
@modulus.main(config_path="conf", config_name="config")
def run(cfg: ModulusConfig) -> None:

    # make list of nodes to unroll graph on
    diff_u1 = Diffusion(T="u_1", D=D1, dim=1, time=False)
    diff_u2 = Diffusion(T="u_2", D=D2, dim=1, time=False)
    diff_in = DiffusionInterface("u_1", "u_2", D1, D2, dim=1, time=False)

    diff_net_u_1 = instantiate_arch(
        input_keys=[Key("x")],
        output_keys=[Key("u_1")],
        cfg=cfg.arch.fully_connected,
    )
    diff_net_u_2 = instantiate_arch(
        input_keys=[Key("x")],
        output_keys=[Key("u_2")],
        cfg=cfg.arch.fully_connected,
    )

    nodes = (
        diff_u1.make_nodes()
        + diff_u2.make_nodes()
        + diff_in.make_nodes()
        + [diff_net_u_1.make_node(name="u1_network", jit=cfg.jit)]
        + [diff_net_u_2.make_node(name="u2_network", jit=cfg.jit)]
    )
```

### A quick note on Hydra to configure Modulus

At the heart of the using Modulus are Hydra config files, the one for this example are found in the code shown below. Using hydra allows a highly customizable but user friendly method for configuring the majority of Modulus’ features. More information can be found in [Modulus Configuration](https://docs.nvidia.com/deeplearning/modulus/text/features/configuration.html).

```yaml
defaults :
  - modulus_default
  - arch:
      - fully_connected
  - scheduler: tf_exponential_lr
  - optimizer: adam
  - loss: sum
  - _self_

arch:
    fully_connected:
        layer_size: 256

save_filetypes : "vtk,npz"

scheduler:
  decay_rate: 0.95
  decay_steps: 100

optimizer: 
  lr : 1e-4

training:
  rec_results_freq: 1000
  max_steps : 5000

batch_size:
  rhs: 2
  lhs: 2
  interface: 2
  interior_u1: 200
  interior_u2: 200
```

## Setting up the Domain: Assigning the boundary and PDE constraints

As described earlier, we need to define a domain for training our neural network. The `Domain` and the configs are passed as inputs when using the `Solver` class. Apart from constraints, you can add various other utilites to the `Domain` such as monitors, validation data, points to do inference on, etc. 

```python
    # make domain add constraints to the solver
    domain = Domain()
```

Now let’s look into adding constraints to this domain. This can be thought of as adding specific constraints to the neural network optimization. For this physics-driven problem, these constraints are the boundary conditions and equation residuals. The goal is to satisfy the boundary conditions exactly, and ideally have the PDE residuals to go 0. These constraints can be specified within Modulus using classes like `PointwiseBoundaryConstrant` and `PointwiseInteriorConstraint`. A L2 loss (defult and can be modified) is then constructed from these constraints which is used by the optimizer to minimize on. Specifying the constraints in this fashion is called soft-constraints.  

$$L = L_{BC} + L_{Residual}$$

**Boundary constraints:** For generating a boundary condition, we need to sample the points on the required boundary/surface of the geometry, specify the nodes we would like to unroll/evaluate on these points and then assign them the desired values. 

A boundary can be sampled using `PointwiseBoundaryConstraint` class. This will sample the entire boundary of the geometry we specify in the `geometry` argument, in this case, both the endpoints of the 1d line. A particular boundary of the geometry can be sub-sampled by using a particular criterion using the `criteria` parameter. For example, to sample the left end of `L1`, `criteria` is set to `Eq(x, 0)`. 

The desired values for the boundary condition are listed as a dictionary in `outvar` argument. For this problem,  we have `'u_1':0` at $x=0$ and `'u_2':100` at $x=2$. At $x=1$, we have the interface condition `'diffusion_interface_dirichlet_u_1_u_2':0` and `'diffusion_interface_neumann_u_1_u_2':0` that we defined earlier (i.e. $U_1=U_2$ and $D_1\frac{dU_1}{dx}=D_2\frac{dU_2}{dx}$). These dictionaries are then used when unrolling the computational graph (specified using the `nodes` argument) for training.

The number of points to sample on each boundary are specified using the `batch_size` parameter. 

**Equations to solve:** The Diffusion PDE we defined is enforced on all the points in the
interior of both the bars, `L1` and `L2`. We will use `PointwiseInteriorConstraint` class to sample points in the interior of the geometry. Again, the appropriate geometry is specified in the `geometry` argument; the equations to solve are specified as a dictionary input to `outvar` argument. These dictionaries are then used when unrolling the computational graph (specified using the `nodes` argument) for training.

For this problem we have the `'diffusion_u_1':0` and `'diffusion_u_2':0` for bars `L1` and `L2` respectively. The parameter `bounds`, determines the range for sampling the values for variables $x$ and $y$. The optional `lambda` parameter can be used to determine the weights for different losses. In this problem, we weight the loss on each point equally, and hence it is not used (defaults to 1) in this problem.

```python
    # sympy variables
    x = Symbol("x")

    # right hand side (x = 2) Pt c
    rhs = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=L2,
        outvar={"u_2": Tc},
        batch_size=cfg.batch_size.rhs,
        criteria=Eq(x, 2),
    )
    domain.add_constraint(rhs, "right_hand_side")

    # left hand side (x = 0) Pt a
    lhs = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=L1,
        outvar={"u_1": Ta},
        batch_size=cfg.batch_size.lhs,
        criteria=Eq(x, 0),
    )
    domain.add_constraint(lhs, "left_hand_side")

    # interface 1-2
    interface = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=L1,
        outvar={
            "diffusion_interface_dirichlet_u_1_u_2": 0,
            "diffusion_interface_neumann_u_1_u_2": 0,
        },
        batch_size=cfg.batch_size.interface,
        criteria=Eq(x, 1),
    )
    domain.add_constraint(interface, "interface")

    # interior 1
    interior_u1 = PointwiseInteriorConstraint(
        nodes=nodes,
        geometry=L1,
        outvar={"diffusion_u_1": 0},
        bounds={x: (0, 1)},
        batch_size=cfg.batch_size.interior_u1,
    )
    domain.add_constraint(interior_u1, "interior_u1")

    # interior 2
    interior_u2 = PointwiseInteriorConstraint(
        nodes=nodes,
        geometry=L2,
        outvar={"diffusion_u_2": 0},
        bounds={x: (1, 2)},
        batch_size=cfg.batch_size.interior_u2,
    )
    domain.add_constraint(interior_u2, "interior_u2")
```

## Adding Validation Data

For this 1d bar problem where the conductivity is constant in each bar, the temperature varies linearly with position inside the solid. The analytical solution can then be given as:

$$
\begin{align}
U_1 = xT_b + (1-x)T_a, && \text{when } 0 \leq x \leq 1 \\
U_2 = (x-1)T_c + (2-x)T_b, && \text{when } 1 \leq x \leq 2 \\
\end{align}
$$

where, 
$$
\begin{align}
T_a = U_1|_{x=0}, && T_c = U_2|_{x=2}, && \frac{\left(T_c + \left( D_1/D_2 \right)T_a \right)}{1+ \left( D_1/D_2 \right)}\\
\end{align}
$$

Now let's create the validators. The validation data is added to the domain using the `PointwiseValidator` class. We use numpy to solve for the `u_1` and `u_2` based on the analytical expressions we showed above. The dictionary of generated numpy arrays (`invar_numpy` and `outvar_numpy`) for input and output variables and the appropriate nodes are used in the definition of the constructor. 

```python
    # validation data
    x = np.expand_dims(np.linspace(0, 1, 100), axis=-1)
    u_1 = x * Tb + (1 - x) * Ta
    invar_numpy = {"x": x}
    outvar_numpy = {"u_1": u_1}
    val = PointwiseValidator(nodes=nodes,invar=invar_numpy, true_outvar=outvar_numpy)
    domain.add_validator(val, name="Val1")

    # make validation data line 2
    x = np.expand_dims(np.linspace(1, 2, 100), axis=-1)
    u_2 = (x - 1) * Tc + (2 - x) * Tb
    invar_numpy = {"x": x}
    outvar_numpy = {"u_2": u_2}
    val = PointwiseValidator(nodes=nodes, invar=invar_numpy, true_outvar=outvar_numpy)
    domain.add_validator(val, name="Val2")
```

## Adding Monitors 

Modulus library allows you to monitor desired quantities in Tensorboard as the simulation progresses and
assess the convergence. A `PointwiseMonitor` can be used to create such an feature. Examples of such quantities can be point values of variables, surface averages, volume averages or any derived quantities that can be formed using the variables being solved.

The variables are available as PyTorch tensors. We can perform tensor operations available in PyTorch to compute any desired derived quantity of our choice. 

In the code below, we create monitors for flux at the interface. The variable `u_1__x` represents the derivative of `u_1` in x-direction (two underscores (`__`) and the variable (`x`)). The same notation is used while handling other derivatives using the Modulus library. (eg. a neumann boundary condition of $\frac{dU_1}{dx}=0$ can be assigned as `'u_1__x':0` in the train domain for solving the same problem with a adiabatic/fixed flux boundary condition). 

The points to sample can be selected in a similar way as we did for specifying some of the interior constraints.

```python
    # make monitors
    invar_numpy = {"x": [[1.0]]}
    monitor = PointwiseMonitor(
        invar_numpy,
        output_names=["u_1__x"],
        metrics={"flux_u1": lambda var: torch.mean(var["u_1__x"])},
        nodes=nodes,
        requires_grad=True,
    )
    domain.add_monitor(monitor)

    monitor = PointwiseMonitor(
        invar_numpy,
        output_names=["u_2__x"],
        metrics={"flux_u2": lambda var: torch.mean(var["u_2__x"])},
        nodes=nodes,
        requires_grad=True,
    )
    domain.add_monitor(monitor)
```

## Putting everything together: Solver and training

We can create a solver by using the domain we just created along with the other configurations that define the optimizer choices, settings (i.e. conf) using Modulus’ `Solver` class. The solver can then be executed using the `solve` method. 

```python
    # make solver
    slv = Solver(cfg, domain)

    # start solver
    slv.solve()


if __name__ == "__main__":
    run()
```

Awesome! We have just completed the file set up for the problem using the Modulus library. We are now ready to solve the PDEs using Neural Networks!

Before we can start training, we can make use of Tensorboard for visualizing the loss values and convergence of several other monitors we just created. This can be done inside the jupyter framework by selecting the directory in which the checkpoint will be stored by clicking on the small checkbox next to it. The option to launch a Tensorboard then shows up in that directory. 

<img src="image_tensorboard.png" alt="Drawing" style="width: 900px;"/>

Also, as Modulus uses Hydra for config management, this causes issues when the code is directly executed through the jupyter notebook. Hence, we will save the code in form of a python script and execute that script inside the jupyter cell. This example is already saved for you and the code block below executes that script [`diffusion_bar.py`](../../source_code/diffusion_1d/diffusion_bar.py). You are encouraged to open the script and go through the code once before executing. Also, feel free to edit the parameters in the [`config.yaml`](../../source_code/diffusion_1d/conf/) file of the model and see its effect on the results.

In [ ]:
import os
os.environ["RANK"]="0"
os.environ["WORLD_SIZE"]="1"
os.environ["MASTER_ADDR"]="localhost"

In [ ]:
!python3 ../../source_code/diffusion_1d/diffusion_bar.py

## Visualizing the solution

Modulus saves the data in several formats (including .vtp, .vtk and .npz). The .npz arrays can be plotted to visualize the output of the simulation. The .npz files that are created are found in the `outputs/` directory. 

Now let's plot the temperature along the bar for the analytical and the neural network solution. A sample script to plot the results is shown below. If the training is complete, you should get the results like shown below. As we can see, our neural network solution and the analytical solution match almost exactly for this diffusion problem. 

<img src="image_diffusion_problem_bootcamp.png" alt="Drawing" style="width: 500px;"/>

In [ ]:
%%capture
import sys
!{sys.executable} -m pip install ipympl
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

plt.figure()

network_dir = "./outputs/diffusion_bar/validators/"
data_1 = np.load(network_dir + "Val1.npz", allow_pickle=True)
data_2 = np.load(network_dir + "Val2.npz", allow_pickle=True)
data_1 = np.atleast_1d(data_1.f.arr_0)[0]
data_2 = np.atleast_1d(data_2.f.arr_0)[0]

plt.plot(data_1["x"][:, 0], data_1["pred_u_1"][:, 0], "--", label="u_1_pred")
plt.plot(data_2["x"][:, 0], data_2["pred_u_2"][:, 0], "--", label="u_2_pred")
plt.plot(data_1["x"][:, 0], data_1["true_u_1"][:, 0], label="u_1_true")
plt.plot(data_2["x"][:, 0], data_2["true_u_2"][:, 0], label="u_2_true")

plt.legend()
plt.savefig("image_diffusion_problem_bootcamp")

# Parameterizing the PDE

As we discussed in the introductory notebook, one important advantage of a PINN solver over traditional numerical methods is its ability to solve parameterized geometries and PDEs. This was initially proposed in the [paper](https://arxiv.org/abs/1906.02382) published by Sun et al. This allows us significant computational advantage, as one can now use PINNs to solve for multiple designs/cases in a single training. Once the training is complete, it is possible to run inference on several geometry/physical parameter combinations as a post-processing step, without solving the forward problem again. 

To demonstrate the concept, we will train the same 1d diffusion problem, but now by parameterizing the conductivity of the first bar in the range (5, 25). Once the training is complete, we can obtain the results for any conductivity value in that range saving us the time to train multiple models. 

## Case Setup

The definition of equations remain the same for this part. Since earlier while defining the equations, we already defined the constants and coefficients of the PDE to be either numerical values or strings, this will allow us to paramterize `D1` by passing it as a string while calling the equations and making the neural network. Now let's start by creating the paramterized train domain. We will skip the parts that are common to the previous section and only discuss the changes. The complete script can be referred in [`diffusion_bar_parameterized.py`](../../source_code/diffusion_1d/diffusion_bar_parameterized.py). 

## Adding Parameterized PDE and Neural Network nodes

Before starting out to create the nodes and domain, we will create the symbolic variable for the $D_1$ and also specify the range of variation for the variable. While the simulation runs, we will validate it against the same diffusion coefficient that we solved earlier i.e. $D_1=10$. 

```python
# params for domain
L1 = Line1D(0, 1)
L2 = Line1D(1, 2)

D1 = Symbol("D1")
D1_range = {D1: (5, 25)}
D1_validation = 1e1

D2 = 1e-1

Tc = 100
Ta = 0
Tb = (Tc + (D1 / D2) * Ta) / (1 + (D1 / D2))
Tb_validation = float(Tb.evalf(subs={D1: 1e1}))

print(Ta)
print(Tb)
print(Tc)
```

For training the parameterized model, we will have the symbolic parameters we just defined as inputs to both the neural networks in `diff_net_u_1` and `diff_net_u_2` viz. `'D1'` along with the usual x coordinate. The outputs remain the same as what we would have for any other non-parameterized simulation.

```python
@modulus.main(config_path="conf", config_name="config_param")
def run(cfg: ModulusConfig) -> None:
    # make list of nodes to unroll graph on
    diff_u1 = Diffusion(T="u_1", D="D1", dim=1, time=False)
    diff_u2 = Diffusion(T="u_2", D=D2, dim=1, time=False)
    diff_in = DiffusionInterface("u_1", "u_2", "D1", D2, dim=1, time=False)

    diff_net_u_1 = instantiate_arch(
        input_keys=[Key("x"), Key("D1")],
        output_keys=[Key("u_1")],
        cfg=cfg.arch.fully_connected,
    )
    diff_net_u_2 = instantiate_arch(
        input_keys=[Key("x"), Key("D1")],
        output_keys=[Key("u_2")],
        cfg=cfg.arch.fully_connected,
    )

    nodes = (
        diff_u1.make_nodes()
        + diff_u2.make_nodes()
        + diff_in.make_nodes()
        + [diff_net_u_1.make_node(name="u1_network", jit=cfg.jit)]
        + [diff_net_u_2.make_node(name="u2_network", jit=cfg.jit)]
    )
```

## Adding Parameterized boundary and PDE Constraints

This part of the code is very similar to the non-parameterized version. The sybolic variables and the ranges that we described earlier need to be inputted to the `param_ranges` attribute of each boundary and internal constraints (`PointwiseBoundaryConstraint` and `PointwiseInteriorConstraint`)

```python
    # make domain add constraints to the solver
    domain = Domain()

    # sympy variables
    x = Symbol("x")

    # right hand side (x = 2) Pt c
    rhs = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=L2,
        outvar={"u_2": Tc},
        batch_size=cfg.batch_size.rhs,
        criteria=Eq(x, 2),
        parameterization=Parameterization(D1_range),
    )
    domain.add_constraint(rhs, "right_hand_side")

    # left hand side (x = 0) Pt a
    lhs = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=L1,
        outvar={"u_1": Ta},
        batch_size=cfg.batch_size.lhs,
        criteria=Eq(x, 0),
        parameterization=Parameterization(D1_range),
    )
    domain.add_constraint(lhs, "left_hand_side")

    # interface 1-2
    interface = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=L1,
        outvar={
            "diffusion_interface_dirichlet_u_1_u_2": 0,
            "diffusion_interface_neumann_u_1_u_2": 0,
        },
        batch_size=cfg.batch_size.interface,
        criteria=Eq(x, 1),
        parameterization=Parameterization(D1_range),
    )
    domain.add_constraint(interface, "interface")

    # interior 1
    interior_u1 = PointwiseInteriorConstraint(
        nodes=nodes,
        geometry=L1,
        outvar={"diffusion_u_1": 0},
        bounds={x: (0, 1)},
        batch_size=cfg.batch_size.interior_u1,
        parameterization=Parameterization(D1_range),
    )
    domain.add_constraint(interior_u1, "interior_u1")

    # interior 2
    interior_u2 = PointwiseInteriorConstraint(
        nodes=nodes,
        geometry=L2,
        outvar={"diffusion_u_2": 0},
        bounds={x: (1, 2)},
        batch_size=cfg.batch_size.interior_u2,
        parameterization=Parameterization(D1_range),
    )
    domain.add_constraint(interior_u2, "interior_u2")
```

## Adding Validators and Monitors 

The process to create these domains is again similar to the previous section. For validation data, we need to create an additional key for the string `'D1'` in the `invar_numpy`. The value for this key can be in the range we specified earlier and which we would like to validate against. It is possible to create multiple validators if required, eg. different $D_1$ values. For monitor domain, similar `invar_numpy` is generated that has both the `'x'` and `'D1'` keys and appropriate arrays. 

```python
    # validation data
    x = np.expand_dims(np.linspace(0, 1, 100), axis=-1)
    u_1 = x * Tb_validation + (1 - x) * Ta
    invar_numpy = {"x": x}
    invar_numpy.update({"D1": np.full_like(invar_numpy["x"], D1_validation)})
    outvar_numpy = {"u_1": u_1}
    val = PointwiseValidator(nodes=nodes,invar=invar_numpy, true_outvar=outvar_numpy)
    domain.add_validator(val, name="Val1")
    
    # make validation data line 2
    x = np.expand_dims(np.linspace(1, 2, 100), axis=-1)
    u_2 = (x - 1) * Tc + (2 - x) * Tb_validation
    invar_numpy = {"x": x}
    invar_numpy.update({"D1": np.full_like(invar_numpy["x"], D1_validation)})
    outvar_numpy = {"u_2": u_2}
    val = PointwiseValidator(nodes=nodes, invar=invar_numpy, true_outvar=outvar_numpy)
    domain.add_validator(val, name="Val2")
    
    # make monitors
    invar_numpy = {"x": [[1.0]], "D1": [[D1_validation]]}
    monitor = PointwiseMonitor(
        invar_numpy,
        output_names=["u_1__x"],
        metrics={"flux_u1": lambda var: torch.mean(var["u_1__x"])},
        nodes=nodes,
        requires_grad=True,
    )
    domain.add_monitor(monitor)

    monitor = PointwiseMonitor(
        invar_numpy,
        output_names=["u_2__x"],
        metrics={"flux_u2": lambda var: torch.mean(var["u_2__x"])},
        nodes=nodes,
        requires_grad=True,
    )
    domain.add_monitor(monitor)
```

## Solver and Training

Now that we have the definitions for the various constraints and domains complete, we can form the solver and run the problem similar to before. The code to do the same can be found below.

```python
    # make solver
    slv = Solver(cfg, domain)

    # start solver
    slv.solve()


if __name__ == "__main__":
    run()
```

In [ ]:
!python3 ../../source_code/diffusion_1d/diffusion_bar_parameterized.py

## Visualizing the solution

The .npz arrays can be plotted similar to previous section to visualize the output of the simulation. You can see that we get the same answer as the analytical solution. You can try to run the problem in `eval` mode by chaning the validation data and see how it performs for the other `D1` values as well. To run the model in evaluation mode (i.e. without training), you just need to [modify the config file](https://docs.nvidia.com/deeplearning/modulus/text/features/configuration.html#run-modes).  

<img src="image_diffusion_problem_bootcamp_parameterized.png" alt="Drawing" style="width: 500px;"/>

You can see that at a fractional increase in computational time, we solved the PDE for $D_1$ ranging from (5, 25). This concept can easily be extended to more complicated problems and this ability of solving parameterized problems comes very handy during desing optimization and exploring the design space. For more examples of solving parameterized problems, please refer to [Modulus User Documentation](https://docs.nvidia.com/deeplearning/modulus/index.html)

# Licensing
This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0).